## Defining the Gaussian filter based on the size of Sigma(Standard Deviation)
<img src="markdown/gaussian.png">

### Reducing noise

In [ ]:
    def gaussian_kernel(self, size, sigma=1):
        size = int(size) // 2
        x, y = np.mgrid[-size:size+1, -size:size+1]
        normal = 1 / (2.0 * np.pi * sigma**2)
        g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
        return g

## Sobel Filter (Edge Detection)
#### Find the regions with sharp change in intensity or color in the image
#### Two seperate filters for the X & Y directions
<img src="markdown/sobel_filters.png">

##### Find The magnitude of the gradient and the angle of the edge
<img src="markdown/gradient.png">

In [ ]:
     def sobel_filters(self, img):
            Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
            Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

            Ix = ndimage.filters.convolve(img, Kx)
            Iy = ndimage.filters.convolve(img, Ky)

            G = np.hypot(Ix, Iy)
            G = G / G.max() * 255
            theta = np.arctan2(Iy, Ix)
            return (G, theta)

# Canny Edge Detection
### Thinning the edge to 1 px by supressing non-max values
#### For ever pixel, find out if it's a local maxima considering the orientation of the edge (find if x bigger than it's neighbors across the edge), then create a thin edge in the center of the gradient
<img src="markdown/non-max.png">


In [ ]:
    def non_max_suppression(self, img, D):
            M, N = img.shape
            Z = np.zeros((M,N), dtype=np.int32)
            angle = D * 180. / np.pi
            angle[angle < 0] += 180


            for i in range(1,M-1):
                for j in range(1,N-1):
                    try:
                        q = 255
                        r = 255

                       #angle 0
                        if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                            q = img[i, j+1]
                            r = img[i, j-1]
                        #angle 45
                        elif (22.5 <= angle[i,j] < 67.5):
                            q = img[i+1, j-1]
                            r = img[i-1, j+1]
                        #angle 90
                        elif (67.5 <= angle[i,j] < 112.5):
                            q = img[i+1, j]
                            r = img[i-1, j]
                        #angle 135
                        elif (112.5 <= angle[i,j] < 157.5):
                            q = img[i-1, j-1]
                            r = img[i+1, j+1]

                        if (img[i,j] >= q) and (img[i,j] >= r):
                            Z[i,j] = img[i,j]
                        else:
                            Z[i,j] = 0


                    except IndexError as e:
                        pass

            return Z

## Hysteresis Thresholding
#### keeping only the dominant edges by removing the weak respnses from the non-max supressor (mostly noise)
#### Define a high threshold and a low threshold, if the edge > the high threshold it'll be kept (strong edge). if the edge < the low threshold it'll be disregarded(weak edge). If the edge falls between the high and the low (High > edge > Low) it'll only be kept if it's connected to a strong edge.

<img src='markdown/Hysteresis.png'>

A is kept since it's higher than the high threshold

D is disregarded since it's lower than the low threshold

C is kept since it's connected to B

E is disregarded since it's not connected to a strong edge

In [ ]:
    def threshold(self, img):

        highThreshold = img.max() * self.highThreshold;
        lowThreshold = highThreshold * self.lowThreshold;

        M, N = img.shape
        res = np.zeros((M,N), dtype=np.int32)

        weak = np.int32(self.weak_pixel)
        strong = np.int32(self.strong_pixel)

        strong_i, strong_j = np.where(img >= highThreshold)
        zeros_i, zeros_j = np.where(img < lowThreshold)

        weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))

        res[strong_i, strong_j] = strong
        res[weak_i, weak_j] = weak

        return (res)

    def hysteresis(self, img):

        M, N = img.shape
        weak = self.weak_pixel
        strong = self.strong_pixel

        for i in range(1, M-1):
            for j in range(1, N-1):
                if (img[i,j] == weak):
                    try:
                        if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                            or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                            or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                            img[i, j] = strong
                        else:
                            img[i, j] = 0
                    except IndexError as e:
                        pass

        return img